# Bài thực hành 1.3: Phát Hiện Làn Đường

Chào mừng đến với bài thực hành 1.3. Trong bài thực hành này, chúng ta sẽ học cách áp dụng thuật toán Canny để phát hiện làn đường cho các phương tiện tự hành.

## Hướng Dẫn

Dưới đây là các hướng dẫn chi tiết về cách lập trình và chạy một phương tiện theo dõi làn đường.

### Thư viện

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.image as mpimage
import matplotlib.pyplot as plt

### Hiệu chuẩn


In [ ]:
import glob
import numpy as np
import cv2
import matplotlib.image as mpimage


class CameraCalibration():
    """ Camera calibration using chessboard images """

    def __init__(self, image_dir, nx, ny):
        """ Initialize the CameraCalibration class.

        Parameters:
            image_dir (str): Path to the directory containing chessboard images
            nx (int): Number of squares in the x direction of the chessboard
            ny (int): Number of squares in the y direction of the chessboard
        """
        # Get the names of all images in the directory
        fnames = glob.glob("{}/*".format(image_dir))

        # List of 3D points for the chessboard corners
        objpoints = []
        # List of 2D points for the chessboard corners in the images
        imagepoints = []

        # Create coordinates for 3D points of the chessboard corners
        objp = np.zeros((nx*ny, 3), np.float32)
        objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

        # Iterate through all chessboard images
        for f in fnames:
            image = mpimage.imread(f)

            # Find chessboard corners
            ret, corners = cv2.findChessboardCorners(image, (nx, ny))
            # If chessboard corners are found
            if ret:
                # Add the corner coordinates to the list
                imagepoints.append(corners)
                # Add the 3D coordinates to the list
                objpoints.append(objp)

        # Shape of the processed images
        shape = (720, 1280)
        # Camera calibration
        ret, self.mtx, self.dist, _, _ = cv2.calibrateCamera(
            objpoints, imagepoints, shape, None, None)

        # If calibration was unsuccessful
        if not ret:
            raise Exception("Unable to calibrate camera")

    # Correct image distortion
    def undistort(self, image):
        """ Returns an undistorted image.

        Parameters:
            image (np.array): Input image

        Returns:
            Image (np.array): Undistorted image
        """
        return cv2.undistort(image, self.mtx, self.dist, None, self.mtx)

Đáp án bài tập 1

In [ ]:
# Path to the directory containing chessboard images
image_dir = r'..\Other\CameraCalibration'
nx = 9
ny = 6

# Initialize the camera calibration object
calibrator = CameraCalibration(image_dir, nx, ny)

# Check the results
# Path to a chessboard image
example_image = cv2.imread(
    r'..\Other\CameraCalibration\calibration1.jpg')

# Correct the image distortion
undistorted_image = calibrator.undistort(example_image)

# Convert image from BGR (OpenCV) to RGB (matplotlib)
example_image_rgb = cv2.cvtColor(example_image, cv2.COLOR_BGR2RGB)
undistorted_image_rgb = cv2.cvtColor(undistorted_image, cv2.COLOR_BGR2RGB)

# Display the images
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(example_image_rgb)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Undistorted Image')
plt.imshow(undistorted_image_rgb)
plt.axis('off')

plt.show()

### Chuyển Đổi Phép Chiếu Hình Ảnh

In [ ]:
class PerspectiveTransformation:
    """ Transforms images between a front view and a top-down view """

    def __init__(self, src, dst):
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.M_inv = cv2.getPerspectiveTransform(dst, src)

    # Transform the image to a top-down view
    def forward(self, image, image_size=(1280, 720), flags=cv2.INTER_LINEAR):
        """ Convert a front view image to a top-down view

        Parameters:
            image (np.array): Front view image
            image_size (tuple): Size of the output image (width, height)
            flags : INTER_LINEAR

        Returns:
            Image (np.array): Top-down view image
        """
        return cv2.warpPerspective(image, self.M, image_size, flags=flags)

    # Restore the image from a top-down view to the original perspective
    def backward(self, image, image_size=(1280, 720), flags=cv2.INTER_LINEAR):
        """ Convert a top-down view image to a front view

        Parameters:
            image (np.array): Top-down view image
            image_size (tuple): Size of the output image (width, height)
            flags (int): INTER_LINEAR

        Returns:
            Image (np.array): Front view image
        """
        return cv2.warpPerspective(image, self.M_inv, image_size, flags=flags)

Đáp án bài tập 2

In [ ]:
# Path to a lane image
example_image = cv2.imread(
    r"E:\Code\ACE\Lap\Lap3\Test Image\Lane_line.jpg")
example_image = cv2.resize(example_image, (1280, 720))

# Define source points
src_points = np.float32([(430, 550),     # top-left point
                         (260, 650),     # bottom-left point
                         (1020, 650),    # bottom-right point
                         (850, 550)])    # top-right point

# Define destination points
dst_points = np.float32([(100, 0),
                        (100, 720),
                        (1180, 720),
                        (1180, 0)])

# Draw points on the image
image = example_image.copy()
# Draw source points (in blue)
for i, point in enumerate(src_points):
    cv2.circle(image, (int(point[0]), int(point[1])),
               5, (255, 0, 0), -1)  # Draw a filled circle
    cv2.putText(image, str(i+1), (int(point[0])-10, int(point[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Draw destination points (in red)
for i, point in enumerate(dst_points):
    cv2.circle(image, (int(point[0]), int(point[1])),
               5, (0, 0, 255), -1)  # Draw a filled circle
    cv2.putText(image, str(i+1), (int(point[0])-10, int(point[1])-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

# Undistort the image
undistorted_image = calibrator.undistort(example_image)
# Convert the image to a top-down view
transform = PerspectiveTransformation(src_points, dst_points)
transform_image = transform.forward(undistorted_image)

# Display the original image
plt.figure(figsize=(10, 5))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")
plt.axis('off')
plt.show()

# Display the transformed image
plt.figure(figsize=(10, 5))
plt.imshow(cv2.cvtColor(transform_image, cv2.COLOR_BGR2RGB))
plt.title("Top-Down View Image")
plt.axis('off')
plt.show()

### Ngưỡng Hóa



In [ ]:
# Compute relative threshold
def threshold_rel(image, lo, hi):
    # Compute the minimum pixel value in the image
    vmin = np.min(image)
    # Compute the maximum pixel value in the image
    vmax = np.max(image)

    # Calculate the low threshold value based on the percentage 'lo'
    vlo = vmin + (vmax - vmin) * lo
    # Calculate the high threshold value based on the percentage 'hi'
    vhi = vmin + (vmax - vmin) * hi
    # Apply thresholding and convert to uint8 data type
    return np.uint8((image >= vlo) & (image <= vhi)) * 255


def threshold_abs(image, lo, hi):
    """ Define a function for absolute thresholding
        Apply thresholding and convert to uint8 data type
    """
    return np.uint8((image >= lo) & (image <= hi)) * 255


class Thresholding:
    """ Extract relevant pixels in the image """

    def __init__(self):
        pass

    def forward(self, image):
        """ Receive an image and extract all lane marking pixels.

        Parameters:
            image (np.array): Input image

        Returns:
            binary (np.array): Binary image representing all positions of lane marking pixels.
        """
        # Convert the input image to HLS color space
        hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        # Convert the input image to HSV color space
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        # Extract the H (hue) channel from the HLS image
        h_channel = hls[:, :, 0]
        # Extract the S (saturation) channel from the HLS image
        s_channel = hls[:, :, 2]
        # Extract the V (value) channel from the HSV image
        v_channel = hsv[:, :, 2]

        # Check if the lane markings are yellow
        # left_lane = threshold_abs(h_channel, 20, 30)

        # Extract the left lane marking pixels
        left_lane = threshold_rel(v_channel, 0.7, 1.0)

        # Extract the right lane marking pixels
        right_lane = threshold_rel(v_channel, 0.7, 1.0)

        # Combine the thresholded images for the left and right lane markings using a logical OR
        result = left_lane | right_lane

        return result  # Return the binary image with lane marking pixels

Đáp án bài tập 3

In [ ]:
# Path to the lane image
example_image = cv2.imread(
    r"E:\Code\ACE\Lap\Lap3\Test Image\Lane_line.jpg")
example_image = cv2.resize(example_image, (1280, 720))

# Initialize the Thresholding class
thresholding = Thresholding()

# Make a copy of the image for processing
image = example_image.copy()
# Undistort the image
undistort_image = calibrator.undistort(image)
# Transform the image to a bird's-eye view
bird_eye_image = transform.forward(undistort_image)
# Apply thresholding to the bird's-eye view image
thresholding_image = thresholding.forward(bird_eye_image)

# Display the thresholded image
plt.figure(figsize=(10, 5))
plt.imshow(cv2.cvtColor(thresholding_image, cv2.COLOR_BGR2RGB))
plt.title("Lane Marking Image")
plt.axis('off')
plt.show()

In [ ]:
class PerspectiveTransformation:
    def __init__(self):
        self.src = np.float32([(280, 400),
                               (30, 650),
                               (1250, 650),
                               (1000, 400)])

        self.dst = np.float32([(180, 0),
                               (180, 720),
                               (1100, 720),
                               (1100, 0)])

        self.M = cv2.getPerspectiveTransform(self.src, self.dst)
        self.M_inv = cv2.getPerspectiveTransform(self.dst, self.src)

    def forward(self, img, img_size=(1280, 720), flags=cv2.INTER_LINEAR):
        return cv2.warpPerspective(img, self.M, img_size, flags=flags)

    def backward(self, img, img_size=(1280, 720), flags=cv2.INTER_LINEAR):
        return cv2.warpPerspective(img, self.M_inv, img_size, flags=flags)

### Phát hiện vạch kẻ đường

In [ ]:
# Control Command Constants
STRAIGHT = 1
SLOW = 2
LEFT = 3
RIGHT = 4
TURNLEFT = 5
TURNRIGHT = 6
FORWARD = 7
BACKWARD = 8
STANDBY = 9
STOP = 10

commands = [
    "Straight",      # 1
    "Slow",         # 2
    "Left",         # 3
    "Right",        # 4
    "TurnLeft",     # 5
    "TurnRight",    # 6
    "Forward",      # 7
    "Backward",     # 8
    "Standby",      # 9
    "Stop"          # 10
]


# Compute the histogram of the bottom half of the image
def hist(image):
    # Select the bottom half of the image
    bottom_half = image[image.shape[0]//2:, :]
    # Compute the histogram along the horizontal axis
    return np.sum(bottom_half, axis=0)


# Lane line detection class
class LaneLines:
    def __init__(self):
        self.left_fit = None    # Polynomial coefficients for the left lane line
        self.right_fit = None   # Polynomial coefficients for the right lane line
        self.nonzero = None     # Indices of non-zero pixels
        self.nonzerox = None    # X coordinates of non-zero pixels
        self.nonzeroy = None    # Y coordinates of non-zero pixels
        self.dir = []           # List to store the directions of detected lane lines
        self.cmd = STRAIGHT     # Default command

        self.nwindows = 9   # Number of windows to detect lane lines
        # Margin around the previous window to search for lane pixels
        self.margin = 30
        self.minpix = 50    # Minimum number of pixels to recenter the window

    def forward(self, image):
        """ Receive an image and detect lane lines.

        Parameters:
            image (np.array): Binary image containing relevant pixels

        Returns:
            Image (np.array): RGB image with lane line pixels highlighted
        """
        # Extract features from the input image
        self.extract_features(image)
        # Fit polynomial curves to the detected lane line pixels
        return self.fit_poly(image)

    # Calculate pixel indices within a window
    def pixels_in_window(self, center, margin, height):
        """ Return all pixels in a specific window

        Parameters:
            center (tuple): coordinates of the window center
            margin (int): width of the window
            height (int): height of the window

        Returns:
            pixelx (np.array): x coordinates of pixels within the window
            pixely (np.array): y coordinates of pixels within the window
        """
        # Top-left corner of the window
        topleft = (center[0]-margin, center[1]-height//2)
        # Bottom-right corner of the window
        bottomright = (center[0]+margin, center[1]+height//2)

        # Condition to select pixels within the window bounds
        condx = (topleft[0] <= self.nonzerox) & (
            self.nonzerox <= bottomright[0])
        condy = (topleft[1] <= self.nonzeroy) & (
            self.nonzeroy <= bottomright[1])
        # Return coordinates of selected pixels
        return self.nonzerox[condx & condy], self.nonzeroy[condx & condy]

    # Extract features from the binary image
    def extract_features(self, image):
        """ Extract features from a binary image

        Parameters:
            image (np.array): Binary image
        """
        # Store the input image
        self.image = image
        # Calculate the height of each window
        self.window_height = int(image.shape[0]//self.nwindows)

        # Get indices of non-zero pixels in the image
        self.nonzero = image.nonzero()
        self.nonzerox = np.array(self.nonzero[1])
        self.nonzeroy = np.array(self.nonzero[0])

    # Find lane pixels in the windows
    def find_lane_pixels(self, image):
        """ Find lane line pixels from a warped binary image.

        Parameters:
            image (np.array): A warped binary image

        Returns:
            leftx (np.array): x coordinates of left lane line pixels
            lefty (np.array): y coordinates of left lane line pixels
            rightx (np.array): x coordinates of right lane line pixels
            righty (np.array): y coordinates of right lane line pixels
            out_image (np.array): RGB image for displaying results.
        """
        # Ensure the input image is grayscale
        assert (len(image.shape) == 2)
        # Create an image to draw the detected lane line pixels
        out_image = np.dstack((image, image, image))

        # Compute the histogram of the bottom half of the image
        histogram = hist(image)
        # Calculate the midpoint of the histogram
        midpoint = histogram.shape[0]//2
        # Find the peak of the left half of the histogram
        leftx_base = np.argmax(histogram[:midpoint])
        # Find the peak of the right half of the histogram
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Initialize the current x coordinate for the left lane line
        leftx_current = leftx_base
        # Initialize the current x coordinate for the right lane line
        rightx_current = rightx_base
        # Initialize the current y coordinate
        y_current = image.shape[0] + self.window_height//2
        # Initialize lists to store detected lane line pixels
        leftx, lefty, rightx, righty = [], [], [], []

        # Iterate through each window
        for _ in range(self.nwindows):
            # Move the window position up
            y_current -= self.window_height

            # Define the center coordinates of the current window
            center_left = (leftx_current, y_current)
            center_right = (rightx_current, y_current)

            # Find lane line pixels in the current window
            good_left_x, good_left_y = self.pixels_in_window(
                center_left, self.margin, self.window_height)
            good_right_x, good_right_y = self.pixels_in_window(
                center_right, self.margin, self.window_height)

            # Add the detected lane line pixels to the respective lists
            leftx.extend(good_left_x)
            lefty.extend(good_left_y)
            rightx.extend(good_right_x)
            righty.extend(good_right_y)

            # Recenter the window if enough pixels are found
            if len(good_left_x) > self.minpix:
                leftx_current = np.int32(np.mean(good_left_x))
            if len(good_right_x) > self.minpix:
                rightx_current = np.int32(np.mean(good_right_x))

        # Return the detected lane line pixels and the result image
        return leftx, lefty, rightx, righty, out_image

    # Fit polynomial curves to the detected lane line pixels
    def fit_poly(self, image):
        """ Find the lane lines from an image and draw them.

        Parameters:
            image (np.array): A warped binary image

        Returns:
            out_image (np.array): An RGB image with the lane lines drawn on it.
        """
        # Find the lane line pixels
        leftx, lefty, rightx, righty, out_image = self.find_lane_pixels(
            image)
        # Fit polynomial curves to the detected lane line pixels
        if len(lefty) > 1500:
            self.left_fit = np.polyfit(lefty, leftx, 2)
        if len(righty) > 1500:
            self.right_fit = np.polyfit(righty, rightx, 2)

        # Calculate y coordinates to draw the lane lines
        maxy = image.shape[0] - 1
        miny = image.shape[0] // 3
        if len(lefty):
            maxy = max(maxy, np.max(lefty))
            miny = min(miny, np.min(lefty))
        if len(righty):
            maxy = max(maxy, np.max(righty))
            miny = min(miny, np.min(righty))
        ploty = np.linspace(miny, maxy, image.shape[0])

        # Calculate x coordinates to draw the lane lines
        left_fitx = self.left_fit[0]*ploty**2 + \
            self.left_fit[1]*ploty + self.left_fit[2]
        right_fitx = self.right_fit[0]*ploty**2 + \
            self.right_fit[1]*ploty + self.right_fit[2]

        # Draw the lane lines on the output image
        for i, y in enumerate(ploty):
            l = int(left_fitx[i])
            r = int(right_fitx[i])
            y = int(y)
            cv2.line(out_image, (l, y), (r, y), (0, 255, 0))

        return out_image

    # Draw the detected lane lines
    def plot(self, out_image):
        np.set_printoptions(precision=6, suppress=True)
        lR, rR, self.pos = self.measure_curvature()
        # curvature_cmd = "Curvature = {:.0f} m".format(min(lR, rR))

        if (self.pos > 0.1):
            self.cmd = LEFT
        elif (self.pos < -0.1):
            self.cmd = RIGHT
        else:
            self.cmd = STRAIGHT

        cv2.putText(out_image, commands[self.cmd-1], org=(10, 240), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=(255, 255, 255), thickness=2)

        cv2.putText(
            out_image,
            "Vehicle is {:.2f} m away from center".format(self.pos),
            org=(10, 310),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.66,
            color=(255, 255, 255),
            thickness=2)

        return out_image

    # Measure vehicle position and lane curvature

    def measure_curvature(self):
        ym = 30/720  # Meters per pixel in the vertical direction
        xm = 3.7/700  # Meters per pixel in the horizontal direction

        # Copy polynomial coefficients
        left_fit = self.left_fit.copy()
        right_fit = self.right_fit.copy()

        # Calculate y coordinate to evaluate curvature
        y_eval = 700 * ym

        # Calculate the curvature radius of the left lane line
        left_curveR = (
            (1 + (2*left_fit[0] * y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])

        # Calculate the curvature radius of the right lane line
        right_curveR = (
            (1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        # Calculate x coordinates of the lane lines at y=700
        xl = np.dot(self.left_fit, [700**2, 700, 1])
        xr = np.dot(self.right_fit, [700**2, 700, 1])

        # Calculate the vehicle position relative to the center of the lane
        pos = (1280//2 - (xl+xr)//2)*xm

        # Return curvature radius of the left and right lane lines and vehicle position
        return left_curveR, right_curveR, pos

    # Handle control commands based on lane detection

    def handle_lane_detect_command(self, speed_cmd, x_battery, x_motor):
        speed = 0
        alpha = 0

        # If the command is to go straight
        if self.cmd == STRAIGHT:
            alpha = x_motor
            speed = speed_cmd

        # If the command is to turn left
        if self.cmd == LEFT:
            if self.pos >= 0.5:
                alpha = 35 + x_battery
                speed = speed_cmd - 45 + x_battery
            elif self.pos >= 0.4:
                alpha = 30 + x_battery
                speed = speed_cmd - 40 + x_battery
            elif self.pos >= 0.3:
                alpha = 25 + x_battery
                speed = speed_cmd - 30 + x_battery
            elif self.pos >= 0.2:
                alpha = 20 + x_battery
                speed = speed_cmd - 20 + x_battery
            elif self.pos >= 0.1:
                alpha = 15 + x_battery
                speed = speed_cmd - 15 + x_battery

        # If the command is to turn right
        if self.cmd == RIGHT:
            if self.pos <= -0.5:
                alpha = 35 + x_battery
                speed = speed_cmd - 45 + x_battery
            elif self.pos <= -0.4:
                alpha = 30 + x_battery
                speed = speed_cmd - 40 + x_battery
            elif self.pos <= -0.3:
                alpha = 25 + x_battery
                speed = speed_cmd - 30 + x_battery
            elif self.pos <= -0.2:
                alpha = 20 + x_battery
                speed = speed_cmd - 20 + x_battery
            elif self.pos <= -0.1:
                alpha = 15 + x_battery
                speed = speed_cmd - 15 + x_battery
            speed += x_motor
            alpha += x_motor

        # Create a control command string
        command = f"{self.cmd} {speed} {int(alpha)}"

        return command

### Xây Dựng Mô Hình Phát Hiện Vạch Làn Đường



Đáp án bài tập 4

In [ ]:
class LaneDetection:
    def __init__(self):
        self.calibration = CameraCalibration(
            r'..\Other\CameraCalibration', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, image):
        # Create a copy of the input image
        output_image = np.copy(image)
        # Remove lens distortion caused by the camera
        undistort_image = self.calibration.undistort(image)
        # Change the image perspective to a top-down view
        bird_eye_image = self.transform.forward(undistort_image)
        # Apply thresholding to extract lane line pixels
        threshold_image = self.thresholding.forward(bird_eye_image)
        # Detect lane lines
        laneline_image = self.lanelines.forward(threshold_image)
        # Restore the image to its original perspective
        laneline_image = self.transform.backward(laneline_image)

        # Blend the processed image with the original image
        output_image = cv2.addWeighted(output_image, 1, laneline_image, 0.6, 0)
        # Draw additional information (lane direction, curvature, etc.) on the processed image
        output_image = self.lanelines.plot(output_image)
        # Return the final processed image
        return output_image

    # Process the input image and detect lane lines in real-time
    def process_image_rt(self, image):
        output_image = self.forward(image)
        return output_image

### Khai báo các hàm số


In [ ]:
from queue import Queue
lane_detection = LaneDetection()
queue_image = Queue()
MAX_QUEUE_SIZE = 3

SPEED_CMD = 150
X_BATTERY = 0
X_MOTOR = -5

In [ ]:
import socket

IMAGE_RESIZE_DIMS = (1280, 720)     # Image dimensions
END_MARKER = b'\xff\xd9'            # End marker for image data
BUFFER_SIZE = 2048                  # Buffer size


def create_udp_socket_listen(ip, port):
    """Create and bind a UDP socket for receiving data"""
    # Create a UDP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    # Bind the socket to the IP address and port
    sock.bind((ip, port))
    return sock


def create_udp_socket_send():
    """Create and configure a UDP socket for sending data"""
    # Create a UDP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    # Configure the socket for sending data
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, BUFFER_SIZE)
    return sock

### Lấy dữ liệu hình ảnh


In [ ]:
def get_image():
    """Receive images via UDP and place them into a queue."""

    # Create a UDP socket to receive image data
    sock = create_udp_socket_listen(ip_lap, port_lap)
    print(f"Listening on IP {ip_lap}, UDP port {port_lap}")

    # Initialize a bytearray to store received image data
    image_data = bytearray()

    global stop_event
    # Loop continues until a stop signal is received
    while not stop_event.is_set():
        # Receive data from the socket with a specified buffer size
        data, _ = sock.recvfrom(BUFFER_SIZE)
        # Append the received data to the bytearray
        image_data.extend(data)

        # Check if the entire image has been received
        if data.endswith(END_MARKER):
            try:
                # Convert the byte data to a numpy array with dtype uint8
                image_array = np.frombuffer(image_data, dtype=np.uint8)
                # Decode the numpy array into an image using OpenCV
                image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                # Resize the image to the specified dimensions
                image = cv2.resize(image, IMAGE_RESIZE_DIMS)
                # Place the image into the queue
                queue_image.put(image)

                # If the queue size exceeds the maximum limit, remove the oldest image
                if queue_image.qsize() >= MAX_QUEUE_SIZE:
                    queue_image.get()
            except Exception as e:
                # Print an error message if there is an issue with image processing
                print(f"Failed to convert image: {str(e)}")

            # Reinitialize the bytearray for the next image
            image_data = bytearray()

    # Close the socket when stopping
    sock.close()

### Phát hiện làn đường


In [ ]:
import time


def process_image():
    global stop_event
    # Save the current time to calculate FPS
    prev_time = time.time()
    # Count the number of frames processed
    frame_count = 0
    # Variable to store FPS
    fps = 0
    # Previous control command for comparison
    prev_cmd = f"{STOP} {0} {0}"

    # Create a UDP socket for sending data
    sock = create_udp_socket_send()

    # Main loop continues until a stop signal is received
    while not stop_event.is_set():
        try:
            # Get an image from the queue
            image = queue_image.get()

            # Calculate FPS
            frame_count += 1
            tmp_time = time.time() - prev_time
            if tmp_time >= 1:
                fps = frame_count / tmp_time
                prev_time = time.time()
                frame_count = 0

            # Process the image to detect lane lines
            image = lane_detection.process_image_rt(image)
            # Process control commands based on lane detection
            cmd = lane_detection.lanelines.handle_lane_detect_command(
                SPEED_CMD, X_BATTERY, X_MOTOR)

        except Exception as e:
            # If there is an error processing the image, display an error message on the image
            cv2.putText(image, "None", (150, 30),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            # Default control command in case of an error
            cmd = f"{STOP} {0} {0}"

        # Send control command to the vehicle controller
        if cmd != prev_cmd:
            try:
                # Convert control command to byte data
                data_bytes_cmd = cmd.encode()
                # Send data to the camera's IP address and port
                sock.sendto(data_bytes_cmd, (ip_cam, port_cam))
                print(f"Sent UDP command: '{cmd}' to {ip_cam}:{port_cam}")
                # Update the previous control command
                prev_cmd = cmd

            except Exception as e:
                print(f"Error sending UDP command: {e}")

        # Display FPS on the image
        cv2.putText(image, f"FPS: {int(fps)}", (10, 30),
                    cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
        # Display the image with processed information
        cv2.imshow("Detected Image", image)

        # Exit the loop if the 'Esc' key is pressed
        if cv2.waitKey(1) & 0xFF == 27:
            # Send stop command when exiting
            cmd = f"{STOP} {0} {0}"
            data_bytes = cmd.encode()

            try:
                # Send the stop command to the vehicle controller
                sock.sendto(data_bytes, (ip_cam, port_cam))
                print(f"Sent final UDP command: '{
                      cmd}' to {ip_cam}:{port_cam}")
            except Exception as e:
                print(f"Error sending final UDP command: {e}")

            stop_event.set()
            break

    # Close the socket and destroy OpenCV windows
    sock.close()
    cv2.destroyAllWindows()

Đáp án bài tập 5

In [ ]:
import threading
stop_event = threading.Event()

ip_lap = "192.168.109.106"
port_lap = 3000

ip_cam = "192.168.109.105"
port_cam = 3001

# Create two threads
threads = [
    # Thread 1 will execute the get_image function to receive images via UDP and place them into the queue
    threading.Thread(target=get_image,
                     name='GetSendImageThread'),
    # Thread 2 will execute the process_image function to process images and send control commands
    threading.Thread(target=process_image,
                     name='ProcessImageThread'),
]

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()

# Close all OpenCV windows after threads have completed
cv2.destroyAllWindows()